In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, LSTM, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping #, ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read data
traindf = pd.read_csv('../input/carry-passengers-hotspot/taxi_data/train_hire_stats.csv')
print("train_hire_stats:\n",traindf.head())
print(traindf.shape)

testdf = pd.read_csv('../input/carry-passengers-hotspot/taxi_data/test_hire_stats.csv')
print("\ntest_hire_stats:\n",testdf.head())
print(testdf.shape)

In [ ]:
zone = traindf.groupby("Zone_ID")
zone_passenger = []
for i in range(1,26):
    zone_passenger.append(zone.get_group(i)['Hire_count'])
    
print(len(zone_passenger))

In [ ]:
for zone in (6,7,8,11,12,13,16,17,18): # 
    plt.figure(figsize=(20,5))
    plt.xlabel('Time',fontsize=20)
    plt.ylabel('Hire_count',fontsize=20)
    plt.plot(zone_passenger[zone][0:696])
    plt.plot(zone_passenger[zone][696:1440])# 745:1441    

In [ ]:
# Check weekday & workday
# holidays連假的平常日
# workdays連假的補班日

holidays= {'2016-02-08', '2016-02-09', '2016-02-10', '2016-02-11', '2016-02-12','2016-02-29', '2016-04-04', '2016-04-05', '2016-06-09', '2016-06-10', 
'2016-09-15', '2016-09-16', '2016-10-10', '2017-01-02', '2017-01-27', '2017-01-30', '2017-02-01','2017-02-27', '2017-02-28'}
makeupworkdays = {'2016-06-04', '2016-09-10', '2017-02-18'}

In [ ]:
# from datetime import date
# #today = date.today().timetuple().tm_yday
# today = date.today().timetuple()
# print( int(today.tm_mday / 7) + 1 )

In [ ]:
# Declare weekday & workday

isworkday = np.ones((len(traindf),), dtype=int)
weekday = np.ones((len(traindf),), dtype=int)
#day_of_year = np.ones((len(traindf),), dtype=int)
#month_of_year = np.ones((len(traindf),), dtype=int)
#day_of_month = np.ones((len(traindf),), dtype=int)
#week_of_year = np.ones((len(traindf),), dtype=int)
week_of_month = np.ones((len(traindf),), dtype=int)

In [ ]:
# Compute weekday & workday
# weekday()
#0 == Monday, #1 == Tuesday, #2 == Wednesday,  #3 == Thursday,  #4 == Friday,  #5 == Saturday, #6 == Sunday

# from datetime import datetime

# for index, row in traindf.iterrows():    
#     if row['Date'] in holidays: # 平常日的國定假日
#         isworkday[index] = 0
#     else:
#         dd=datetime.strptime(row['Date'], "%Y-%m-%d")
#         weekday[index]= dd.weekday() 
#         if weekday[index] >=5 and row['Date'] not in makeupworkdays: # 非補假日的假日
#             isworkday[index] = 0

from datetime import datetime

for index, row in traindf.iterrows():
    dd=datetime.strptime(row['Date'], "%Y-%m-%d")
    #day_of_year[index] = dd.timetuple().tm_yday
    #month_of_year[index] = dd.timetuple().tm_mon
    #day_of_month[index] = dd.timetuple().tm_mday 
    #week_of_year[index] = int(dd.timetuple().tm_yday / 7) + 1
    week_of_month[index] = int(dd.timetuple().tm_mday / 7) + 1
    weekday[index]= dd.weekday()
    if row['Date'] in holidays: # 平常日的國定假日
        isworkday[index] = 0        
    elif weekday[index] >=5 and row['Date'] not in makeupworkdays:      # 非補假日的假日  
        isworkday[index] = 0

In [ ]:
#Build a new dataframe from the training data

train_data = pd.DataFrame(traindf[["Zone_ID", "Hour_slot","Hire_count"]])
#train_data = pd.DataFrame(traindf)
train_data['isworkday'] = isworkday
train_data['weekday'] = weekday
#train_data['dayofyear'] = day_of_year
#train_data['monthofyear'] = month_of_year
#train_data['dayofmonth'] = day_of_month
#train_data['weekofyear'] = week_of_year
train_data['weekofmonth'] = week_of_month

In [ ]:
train_data_partial = pd.concat([ train_data.loc[8784*6:8784*9-1,:], train_data.loc[8784*11:8784*14-1,:], train_data.loc[8784*16:8784*19-1,:] ])

In [ ]:
# print(train_data.shape)
# print(train_data.head())
print(train_data_partial.shape)
print(train_data_partial.head())
print(train_data_partial.sample(n=5))

In [ ]:
# train_data.loc[3090:3100,['Date']+['isworkday']]
# train_data.loc[3090:3120,:]

In [ ]:
# train_label = train_data["Hire_count"].values
# train_data_nolab = train_data.drop(columns=['Hire_count'])
train_label = train_data_partial["Hire_count"].values
train_data_nolab = train_data_partial.drop(columns=['Hire_count'])
print(train_label.shape)

In [ ]:
# See train data

print(train_data_nolab.head())
print(train_data_nolab.sample(n=5))
print(len(train_label))
print(train_label)

In [ ]:
#Use OneHotEncoder

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

#Build encoder

encoder.fit_transform(train_data_nolab)
encoder.categories_

In [ ]:
#Transform data into one hot vector

train_data_encoded = encoder.transform(train_data_nolab).toarray()
train_data_encoded.shape

# New process

In [ ]:
print(train_data_encoded[5])
print(train_data_encoded[6])

In [ ]:
# divide data by zone

unit_zone_num = int(train_data_encoded.shape[0] / 9)
train_data_encoded_split = []
for i in range(9):
    train_data_encoded_split.append(train_data_encoded[ i * unit_zone_num : (i+1) * unit_zone_num ])

In [ ]:
train_data_encoded_split[0].shape

In [ ]:
previous_hour=47
train_data_combine_zone = []
for zone in range(9):
    train_data_combine = []
    for i in range(previous_hour, train_data_encoded_split[zone].shape[0]):
        combine_data = train_data_encoded_split[zone][i]
        for j in range(1, previous_hour+1):
            combine_data = np.concatenate((train_data_encoded_split[zone][i-j], combine_data),axis=None)
        train_data_combine.append(combine_data)
    train_data_combine_zone.append(train_data_combine)

In [ ]:
np.array(train_data_combine_zone[1]).shape

In [ ]:
# merge 9 area encode combine data

train_data_encoded_combine = np.array(train_data_combine_zone[0])
for zone in range(1,9):
    train_data_encoded_combine = np.concatenate((train_data_encoded_combine, np.array(train_data_combine_zone[zone])),axis=0)

In [ ]:
print(train_data_encoded_combine.shape)
in_dim = train_data_encoded_combine.shape[1]
print(in_dim)

In [ ]:
# train_data_encoded_new = np.array(train_data_encoded_new)
# print(train_data_encoded_new.shape)
# in_dim = train_data_encoded_new.shape[1]
# print(in_dim)

In [ ]:
unit_zone_num_label = int(train_label.shape[0] / 9)
print(unit_zone_num_label)

In [ ]:
print(train_label.shape)
train_label_new = train_label[0 * unit_zone_num_label + previous_hour: (1) * unit_zone_num_label]
for i in range(1,9):
    train_label_new = np.concatenate( (train_label_new, train_label[i*unit_zone_num_label+previous_hour:(i+1)*unit_zone_num_label] ), axis=0)
print(train_label_new.shape)

In [ ]:
# print(train_label.shape)
# train_label=train_label[previous_hour:]
# print(train_label.shape)

In [ ]:
# split into 80% for train and 20% for validation
seed = 2048
training_data, validation_data, training_label, validation_label = train_test_split(train_data_encoded_combine, train_label_new, test_size=0.2, random_state = seed)

In [ ]:
print(training_data.shape)
print(validation_data.shape)
print(training_label.shape)

In [ ]:
# model = Sequential()
# model.add(Dense(512, activation='relu', input_dim = in_dim ))
# model.add(Dropout(0.1))
# #model.add(BatchNormalization())
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.1))
# # model.add(BatchNormalization())
# #model.add(Dropout(0.1))
# model.add(Dense(1))
# model.summary()

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=in_dim))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(1))
model.summary()

model.compile(loss='mse', optimizer=Adam(lr=1e-3,decay=1e-5),
             metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
# callback setting
#checkpoint = ModelCheckpoint(f'/kaggle/working/_s{seed_value}.hdf5', monitor='val_accuracy', verbose=0, save_best_only=True,
#checkpoint = ModelCheckpoint('/kaggle/working/_weights.hdf5', monitor='val_accuracy', verbose=0, save_best_only=True,
#                             save_weights_only=True, mode='max')
earlystop = EarlyStopping(monitor='val_root_mean_squared_error', min_delta=1e-3, patience=15, verbose=1, mode='min', baseline=None,
                          restore_best_weights=False)
# reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=6, verbose=1, min_delta=1e-4, cooldown=2,
#                              min_lr=1e-11, mode='max')

# callbacks_list = [checkpoint, earlystop, reduceLR]

callbacks_list = [earlystop]

train_history = model.fit(training_data, # train_data_encoded_combine
                          training_label, # train_label_new,
                          batch_size=512,
                          epochs=80,
                          validation_data=(validation_data, validation_label), 
                          shuffle=True,
                          verbose=1, 
                          callbacks = callbacks_list)

print("*+"*60)
print('Finish\n')

In [ ]:
def show_train_history(train_history,train, validation): #
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel("Epoch")
    plt.legend(['train','validation'],loc='upper right') # 
    plt.show()

show_train_history(train_history, 'loss', 'val_loss') #
show_train_history(train_history, 'root_mean_squared_error', 'val_root_mean_squared_error')

In [ ]:
#Declare weekday & workday

isworkday_test = np.ones((len(testdf),), dtype=int)
weekday_test = np.ones((len(testdf),), dtype=int)
#day_of_year_test = np.ones((len(testdf),), dtype=int)
#month_of_year_test = np.ones((len(testdf),), dtype=int)
#day_of_month_test = np.ones((len(testdf),), dtype=int)
#week_of_year_test = np.ones((len(testdf),), dtype=int)
week_of_month_test = np.ones((len(testdf),), dtype=int)

In [ ]:
# for index, row in testdf.iterrows():    
#     if row['Date'] in holidays:
#         isworkday_test[index] = 0
#     else:
#         dd=datetime.strptime(row['Date'], "%Y-%m-%d")
#         weekday_test[index]= dd.weekday() 
#         if weekday_test[index] >=5 and row['Date'] not in makeupworkdays:
#             isworkday_test[index] = 0

for index, row in testdf.iterrows():
    dd=datetime.strptime(row['Date'], "%Y-%m-%d")
    #day_of_year_test[index] = dd.timetuple().tm_yday
    #month_of_year_test[index] = dd.timetuple().tm_mon
    #day_of_month_test[index] = dd.timetuple().tm_mday
    #week_of_year_test[index] = int(dd.timetuple().tm_yday / 7) + 1
    week_of_month_test[index] = int(dd.timetuple().tm_mday / 7) + 1
    weekday_test[index]= dd.weekday()
    
    if row['Date'] in holidays: # 平常日的國定假日
        isworkday_test[index] = 0
    elif weekday_test[index] >=5 and row['Date'] not in makeupworkdays: # 非補假日的假日
        isworkday_test[index] = 0       

In [ ]:
test_data = pd.DataFrame(testdf[["Zone_ID", "Hour_slot"]])
test_data['isworkday'] = isworkday_test
test_data['weekday'] = weekday_test
#test_data['dayofyear'] = day_of_year_test
#test_data['monthofyear'] = month_of_year_test
#test_data['dayofmonth'] = day_of_month_test
#test_data['weekofyear'] = week_of_year_test
test_data['weekofmonth'] = week_of_month_test

In [ ]:
print(test_data.shape)
print(test_data.head())
print(test_data.sample(n=5))

# New Process

In [ ]:
# divide data by zone
test_data_split = test_data.groupby("Zone_ID")
train_data_partial_split = train_data_partial.groupby("Zone_ID")

# unit_zone_num_test = int(testing_data.shape[0] / 9)
# test_data_encoded_split = []
# for i in range(9):
#     test_data_encoded_split.append(testing_data[ i * unit_zone_num_test : (i+1) * unit_zone_num_test ])

In [ ]:
len(test_data_split.get_group(7))

In [ ]:
test_data_concate = []
for zone in (7,8,9,12,13,14,17,18,19):  
    test_data_concate.append( pd.concat( (train_data_partial_split.get_group(zone).drop(columns=['Hire_count'])[-previous_hour:], test_data_split.get_group(zone)) ) ) # [-(14*24):]

In [ ]:
print(test_data_concate[0])
print(test_data_concate[0].shape)

In [ ]:
unit_zone_num_test = len(test_data_concate[0])
print(unit_zone_num_test)
print(len(test_data_concate))

In [ ]:
# test_data_concate_all = test_data_concate[0]
# for i in range(1,9):
#     test_data_concate_all = pd.concat( ( test_data_concate_all, test_data_concate[i] ) )

In [ ]:
# print(test_data_concate_all)

In [ ]:
# testing_data = encoder.transform(test_data_concate_all).toarray()
# print(testing_data.shape)

In [ ]:
testing_data_encoded = []
for i in range(9):
    testing_data_encoded.append(encoder.transform(test_data_concate[i]).toarray() )

In [ ]:
testing_data_encoded[1].shape

In [ ]:
test_data_combine_zone = []
for zone in range(9):
    test_data_combine = []
    for i in range(previous_hour, testing_data_encoded[zone].shape[0]):
        combine_data = testing_data_encoded[zone][i]
        for j in range(1,previous_hour+1):
            combine_data = np.concatenate((testing_data_encoded[zone][i-j], combine_data),axis=None)
        test_data_combine.append(combine_data)
    test_data_combine_zone.append(test_data_combine)

In [ ]:
np.array(test_data_combine_zone[1]).shape

In [ ]:
# merge 9 area encode combine data

test_data_encoded_combine = np.array(test_data_combine_zone[0])
for zone in range(1,9):
    test_data_encoded_combine = np.concatenate((test_data_encoded_combine, np.array(test_data_combine_zone[zone])),axis=0)

In [ ]:
print(test_data_encoded_combine.shape)

In [ ]:
# predicts = model.predict(testing_data)
# predicts = np.round(predicts, 0)
predicts = model.predict(test_data_encoded_combine)
predicts = np.round(predicts, 0)

In [ ]:
predicts.shape

In [ ]:
for i in range(len(predicts)):
    if (predicts[i] < 0):
        predicts[i] = 0

In [ ]:
plt.figure(figsize=(20,5))
plt.xlabel('Index',fontsize=20)
plt.ylabel('Hire_count',fontsize=20)
plt.plot(predicts)#[672:1343]

In [ ]:
prediction = testdf
prediction['Hire_count'] = predicts
prediction.to_csv('predict.csv',index=False)

In [ ]:
prediction.head()

In [ ]:
prediction.sample(n=10)

In [ ]:
prediction.describe()